In [ ]:
#Reference Notebook: https://www.kaggle.com/code/srinathsrinivasan1/retinal-oct-using-vgg19-and-mobnetv2-a-comparison

In [1]:
import os
import random

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics

import tensorflow as tf
import tensorflow_addons as tfa
from keras.preprocessing.image import ImageDataGenerator, load_img

In [2]:
base_dir = os.path.join('data/OCT2017 /')
print('Base directory --> ', os.listdir(base_dir))

Base directory -->  ['.DS_Store', 'test', 'train', 'val']


In [3]:
train_dir = os.path.join(base_dir + "train/")
print("Train Directory --> ", os.listdir(train_dir))

Train Directory -->  ['CNV', '.DS_Store', 'DME', 'DRUSEN', 'NORMAL']


In [4]:
test_dir = os.path.join(base_dir + "test/")
print("Test Directory --> ", os.listdir(test_dir))

Test Directory -->  ['CNV', '.DS_Store', 'DME', 'DRUSEN', 'NORMAL']


In [5]:
validation_dir = os.path.join(base_dir + "val/")
print("Validation Directory --> ", os.listdir(test_dir))

Validation Directory -->  ['CNV', '.DS_Store', 'DME', 'DRUSEN', 'NORMAL']


In [6]:
vgg19 = tf.keras.applications.VGG19(
    include_top = False,
    weights = 'imagenet',
    input_tensor = None,
    input_shape = (150,150,3),
    pooling = None,
    classes = 1000
)

Metal device set to: Apple M1 Pro


2022-04-06 21:47:11.019788: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-06 21:47:11.019889: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
vgg19.trainable = False

In [8]:
model_vgg = tf.keras.models.Sequential([

    vgg19,
    tf.keras.layers.Conv2D(128, kernel_size = (3, 3), padding = 'same'),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),# modifying final layers of VGG-19
    tf.keras.layers.Conv2D(64, kernel_size = (3, 3), padding = 'same'),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),
    tf.keras.layers.Dense(4, activation = 'softmax')
])

In [9]:
metrics = ['accuracy',
                tfa.metrics.F1Score(num_classes = 4),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall()]

In [10]:
model_vgg.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = metrics)
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 128)         589952    
                                                                 
 p_re_lu (PReLU)             (None, 4, 4, 128)         2048      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 64)          73792     
                                                                 
 p_re_lu_1 (PReLU)           (None, 4, 4, 64)          1024      
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 100)               1

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 500)

Found 83484 images belonging to 4 classes.


In [12]:
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 16)

Found 32 images belonging to 4 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 50)

Found 968 images belonging to 4 classes.


In [14]:
earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, mode='max', min_delta=0.1)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='data/vgg19_model.h5',
                                                                   save_weights_only=True, monitor='accuracy',
                                                                   mode='max', save_best_only=True)
history_vgg = model_vgg.fit(
    train_generator,
    steps_per_epoch = (83484/500),
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = (32/16),
    max_queue_size=100,
    #workers = 1 ,
    #use_multiprocessing=True,
    verbose = 1,
    callbacks=[model_checkpoint_callback, earlyStopCallback])

Epoch 1/20


2022-04-06 21:47:14.750521: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-06 21:47:15.431126: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 17/166 [==>...........................] - ETA: 7:07 - loss: 1.1958 - accuracy: 0.5549 - f1_score: 0.3202 - precision: 0.6598 - recall: 0.3367

KeyboardInterrupt: 

In [ ]:
print("Values for VGG-19 based ConvNet")
acc = history_vgg.history['accuracy']
val_acc = history_vgg.history['val_accuracy']
loss = history_vgg.history['loss']
val_loss = history_vgg.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(12,12))

plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
plt.title('Training & validation accuracy')
plt.legend()

plt.figure(figsize = (12,12))

plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training $ validation loss')
plt.legend()

plt.show()

In [ ]:
model_vgg.evaluate(test_generator)

In [ ]:
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model_vgg.predict_generator(test_generator, steps = test_steps_per_epoch)

predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

In [ ]:
report = sklearn.metrics.classification_report(true_classes, predicted_classes, target_names = class_labels)
print(report)

In [ ]:
cm = sklearn.metrics.confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8,8))
classes = train_generator.class_indices.keys()
sns.heatmap(cm, fmt='.0f', annot=True, linewidths=0.2, linecolor='purple', xticklabels=classes)
plt.show()